# Stacks and Queues

## Agenda

1. Stacks (last in, first out (LIFO) / first in, last out)
     - ... for delimiter pairing
     - ... for postfix expression evaluation
     - ... for tracking execution and *backtracking*
2. Queues
    - ... for tracking execution and *backtracking*
    - ... for apportioning work
    - ... for fair scheduling (aka "round-robin" scheduling)
3. Run-time analysis
4. Closing remarks

## Overview

While the list data structure is incredibly useful, both implementations we explored (array-backed and linked) have operations that run in $O(N)$ time, which make them non-ideal for use with large, growing collections of data.

By further restricting the list API, however — in particular, by *isolating points of access to either the front or end of the data set* — we can create data structures whose operations are all $O(1)$, and remain very useful in their own right.

## 1. Stacks

Stacks are linear data structures which only permit access to one "end" of the data collection. We can only append ("push") items onto the tail end (a.k.a. the "top") of a stack, and only the most recently added item can be removed ("popped"). The last item to be pushed onto a stack is therefore the first one to be popped off, which is why we refer to stacks as last-in, first out (LIFO) structures.

Constant time run-time performance

In [7]:
# array-backed implementation

class Stack:
    def __init__(self):
        self.data = []
        
    def push(self, val):
        self.data.append(val)
    
    def pop(self):
        assert(self.data)
        rv = self.data[-1]
        del self.data[-1]
        return rv
    
    def empty(self):
        return(len(self.data) == 0)
    
    def __bool__(self):
        return not self.empty()

In [14]:
s = Stack()
for x in range(10):
    s.push(x)

In [15]:
while s:
    print(s.pop())

9
8
7
6
5
4
3
2
1
0


In [10]:
s2 = Stack()
bool(s2)

False

In [31]:
# linked implementation

class Stack:
    class Node:
        def __init__(self, val, next=None):
            self.val = val
            self.next  = next
    
    def __init__(self):
        self.top = None

    def push(self, val):
        self.top = Stack.Node(val,next=self.top)
        
    def pop(self):
        assert(self.top)
        rv = self.top.val
        self.top = self.top.next
        return rv
    
    def empty(self):
        return self.top is None
    
    def __bool__(self):
        return not self.empty()

In [32]:
s = Stack()
for x in range(10):
    s.push(x)

In [33]:
while s:
    print(s.pop())

9
8
7
6
5
4
3
2
1
0


### ... for delimiter pairing

e.g., `'(1 + 2 * (3 - (4 / 2) + 5) - (6 + 1))'`

In [41]:
def check_parens(expr):
    s = Stack()
    for c in expr:
        if c == '(':
            s.push(c)
        elif c == ')':
            try:
                assert s.pop() == '('
            except:
                return False
    return s.empty()

In [42]:
check_parens('()')

True

In [43]:
check_parens('((()))')

True

In [44]:
check_parens('()(()()(()))')

True

In [45]:
check_parens('(')

False

In [46]:
check_parens('())')

False

In [47]:
check_parens('(1 + 2 * (3 - (4 / 2) + 5) - (6 + 1))')

True

### ... for postfix expression (aka "reverse polish notation") evaluation

e.g., `'(1 + 2) * 5'` $\rightarrow$ `'1 2 + 5 *'` (1 ENTER 2 + ENTER 5 *)

In [48]:
def eval_postfix(expr):
    s = Stack()
    toks = expr.split()
    for t in toks:
        if t == '+':
            s.push(s.pop() + s.pop())
        elif t == '*':
            s.push(s.pop() * s.pop())
        else: #number
            s.push(int(t))
    return s.pop()

In [49]:
eval_postfix('1 2 + 5 *')

15

In [50]:
eval_postfix('1 2 5 * +')

11

In [51]:
# ((1 + 2) * (3 + 2)) * 10
eval_postfix('1 2 + 3 2 + * 10 *')

150

In [53]:
#Check is number using _.isdigit()

### ... for tracking execution and backtracking

In [54]:
maze_str = """######   
              I    #   
              # ## #   
              # ####   
              #    O   
              ######"""

def parse_maze(maze_str):
    grid = []
    for line in maze_str.split('\n'):
        grid.append(['# IO'.index(c) for c in line.strip()])
    return grid

def print_maze(grid):
    for r in grid:
        print(''.join('# IO!+'[c] for c in r))

In [55]:
parse_maze(maze_str)

[[0, 0, 0, 0, 0, 0],
 [2, 1, 1, 1, 1, 0],
 [0, 1, 0, 0, 1, 0],
 [0, 1, 0, 0, 0, 0],
 [0, 1, 1, 1, 1, 3],
 [0, 0, 0, 0, 0, 0]]

In [56]:
maze = parse_maze(maze_str)
maze[1][1] = 4
print_maze(maze)

######
I!   #
# ## #
# ####
#    O
######


In [57]:
class Move:
    def __init__(self, frm, to):
        self.frm = frm
        self.to  = to
        
    def __repr__(self):
        return '({},{}) -> ({},{})'.format(self.frm[0], self.frm[1],
                                           self.to[0],  self.to[1])

def moves(maze, place):
    moves = [Move(place, (place[0]+d[0], place[1]+d[1]))
            for d in ((-1, 0), (1, 0), (0, -1), (0, 1))
            if place[0]+d[0] in range(len(maze)) and 
               place[1]+d[1] in range(len(maze[0])) and
               maze[place[0]+d[0]][place[1]+d[1]] in (1, 2, 3)]
    return moves

In [58]:
from time import sleep
from IPython.display import clear_output

def visit(maze, loc):
    """Annotates a loc in the maze as visited"""
    maze[loc[0]][loc[1]] = 5
    
def mark(maze, loc):
    """Annotates a loc in the maze as being of interest"""
    if maze[loc[0]][loc[1]] != 3:
        maze[loc[0]][loc[1]] = 4
    
def display(maze):
    clear_output(True)
    print_maze(maze)
    sleep(0.5)

def solve_maze(maze, entry):
    for m in moves(maze, entry):
        save_move(m)
    visit(maze, entry)
    while not out_of_moves():
        move = next_move()
        if maze[move.to[0]][move.to[1]] == 3:
            return True
        display(maze)
        visit(maze, move.to)
        for m in moves(maze, move.to):
            mark(maze, m.to)
            save_move(m)
    return False

In [61]:
move_stack = Stack()

def save_move(move):
    move_stack.push(move)

def next_move():
    return move_stack.pop()

def out_of_moves():
    return move_stack.empty()

In [63]:
solve_maze(parse_maze(maze_str), (1, 0))

######
+++++#
#+##+#
#+####
#+++!O
######


True

In [64]:
maze_str = """#################
              I #       #     #
              # ##### # # # # #
              #     # # # # # #
              # ### ### # # ###
              #   #       #   O
              #################"""

In [65]:
solve_maze(parse_maze(maze_str), (1, 0))

#################
++#       #+++++#
#+##### # #+#+#+#
#+++++# # #+#+#+#
#!###+###!#+#+###
#   #+++++++#++!O
#################


True

In [66]:
maze_str = """#################
              I               #
              # # # # # # # # #
              # # # # # # # # #
              # ###############
              #               O
              #################"""

In [67]:
solve_maze(parse_maze(maze_str), (1, 0))

#################
++++++++++++++++#
#+#+#+#+#+#+#+#+#
#+#+#+#+#+#+#+#+#
#+###############
#++++++++++++++!O
#################


True

"Depth first" search: natural type of seach when using stack to save moves

## 2. Queues

Queues are linear data structures wherein we are only permitted to append ("enqueue") items onto the rear, and remove ("dequeue") items from the front. The oldest item still in a queue is therefore the next one to be dequeued, which is why we refer to a queue as a first-in, first-out (FIFO) structure. It is helpful to think of a queue as being the model for a line at a typical supermarket checkout aisle (first customer in, first customer to be checked out).

In [92]:
# array-backed implementation

class Queue:
    def __init__(self):
        self.data = []
        self.head = 0

    def enqueue(self, val): #O(1)
        self.data.append(val)
        
    def dequeue(self): #O(N)
        assert not self.empty()
        rv = self.data[self.head]
        self.head += 1
        #del self.data[0]
        return rv
    
    def empty(self):
        return self.head == len(self.data)
    
    def __bool__(self):
        return not self.empty()

In [93]:
q = Queue()
for x in range(10):
    q.enqueue(x)

In [94]:
while q:
    print(q.dequeue())

0
1
2
3
4
5
6
7
8
9


In [95]:
q2 = Queue()
for x in range(100):
    q2.enqueue(x)
    q2.dequeue()

In [98]:
len(q2.data)

100

In [104]:
# linked implementation

class Queue:
    class Node:
        def __init__(self, val, next=None):
            self.val = val
            self.next  = next
    
    def __init__(self):
        self.head = self.tail = None

    def enqueue(self, val): #O(1)
        if self.tail:
            self.tail.next = self.tail = Queue.Node(val)
        else:
            self.head = self.tail = Queue.Node(val)
    
    def dequeue(self): #O(1)
        assert(self.head)
        rv = self.head.val
        self.head = self.head.next
        if not self.head:
            self.tail = None
        return rv
    
    def empty(self):
        return self.tail is None
    
    def __bool__(self):
        return not self.empty()

In [105]:
q = Queue()
for x in range(10):
    q.enqueue(x)

In [106]:
while q:
    print(q.dequeue())

0
1
2
3
4
5
6
7
8
9


### ... for tracking execution and backtracking

In [107]:
move_queue = Queue()

def save_move(move):
    move_queue.enqueue(move)

def next_move():
    return move_queue.dequeue()

def out_of_moves():
    return move_queue.empty()

In [108]:
maze_str = """######   
              I    #   
              # ## #   
              # ####   
              #    O   
              ######"""

In [109]:
solve_maze(parse_maze(maze_str), (1, 0))

######
+++++#
#+##+#
#+####
#+++!O
######


True

In [110]:
maze_str = """#################
              I #       #     #
              # ##### # # # # #
              #     # # # # # #
              # ### ### # # ###
              #   #       #   O
              #################"""

In [111]:
solve_maze(parse_maze(maze_str), (1, 0))

#################
++#+++++++#+++++#
#+#####+#+#+#+#+#
#+++++#+#+#+#+#+#
#+###+###+#+#+###
#+++#+++++++#++!O
#################


True

In [112]:
maze_str = """#################
              I               #
              # # # # # # # # #
              # # # # # # # # #
              # ###############
              #               O
              #################"""

In [113]:
solve_maze(parse_maze(maze_str), (1, 0))

#################
++++++++++++++++#
#+#+#+#+#+#+#+#+#
#+#+#+#+#+#+#+#+#
#+###############
#++++++++++++++!O
#################


True

"breadth-first" search: good for when other possibilities have infinite routes

### ... for doling out work

In [122]:
from threading import Thread, Lock
from time import sleep
import random

lock = Lock()
def worker_fn(cid, q):
    while True:
        try:
            with lock:
                work = q.dequeue()
        except: # queue is empty
            sleep(1)
            continue
        if work == 'Stop':
            print('Consumer', cid, 'stopping.')
            break
        else:
            print('Consumer', cid, 'processing', work)
            sleep(random.random())

work_queue = Queue()
for i in range(5):
    Thread(target=worker_fn, args=(i, work_queue)).start()

In [123]:
import threading
threading.active_count()

10

In [124]:
for i in range(10):
    with lock:
        work_queue.enqueue(i)

Consumer 0 processing 0
Consumer 1 processing 1
Consumer 2 processing 2
Consumer 3 processing 3
Consumer 4 processing 4
Consumer 3 processing 5
Consumer 0 processing 6
ConsumerConsumer 4 processing 8
 0 processing 7
Consumer 1 processing 9


In [125]:
for i in range(5):
    with lock:
        work_queue.enqueue('Stop')

Consumer 2 stopping.
Consumer 4 stopping.
Consumer 0 stopping.
Consumer 3 stopping.
Consumer 1 stopping.


### ... for fair scheduling (aka "round-robin" scheduling)

In [118]:
from random import randint
from time import sleep

task_queue = Queue()
for i in range(3):
    task_queue.enqueue(('Job {}'.format(i), randint(3, 6)))

In [119]:
n = task_queue.head
while n:
    print(n.val)
    n = n.next

('Job 0', 6)
('Job 1', 4)
('Job 2', 6)


In [120]:
while not task_queue.empty():
    job, time_left = task_queue.dequeue()
    print('Running', job)
    sleep(1)
    time_left -= 1
    if time_left > 0:
        print('Re-queueuing', job, 'with remaining time =', time_left)
        task_queue.enqueue((job, time_left))
    else:
        print('*', job, 'done')

Running Job 0
Re-queueuing Job 0 with remaining time = 5
Running Job 1
Re-queueuing Job 1 with remaining time = 3
Running Job 2
Re-queueuing Job 2 with remaining time = 5
Running Job 0
Re-queueuing Job 0 with remaining time = 4
Running Job 1
Re-queueuing Job 1 with remaining time = 2
Running Job 2
Re-queueuing Job 2 with remaining time = 4
Running Job 0
Re-queueuing Job 0 with remaining time = 3
Running Job 1
Re-queueuing Job 1 with remaining time = 1
Running Job 2
Re-queueuing Job 2 with remaining time = 3
Running Job 0
Re-queueuing Job 0 with remaining time = 2
Running Job 1
* Job 1 done
Running Job 2
Re-queueuing Job 2 with remaining time = 2
Running Job 0
Re-queueuing Job 0 with remaining time = 1
Running Job 2
Re-queueuing Job 2 with remaining time = 1
Running Job 0
* Job 0 done
Running Job 2
* Job 2 done


## 3. Run-time analysis

Stack & Queue implementations:

- Insertion (push and enqueue) = $O(1)$
- Deletion (pop and dequeue) = $O(1)$

## 4. Closing remarks